In [17]:
import pandas as pd
import sys
import itertools
import numpy as np
import random
import os
import requests
from tqdm import tqdm
import glob
import json


In [2]:
!ls -la 'IGSR/23andme_rel_ped.chip.omni_broad_sanger_combined.20140818.snps.genotypes.h5'


-rw-r--r-- 1 jbonnell kmem 3918820750 Jul 19 16:28 IGSR/23andme_rel_ped.chip.omni_broad_sanger_combined.20140818.snps.genotypes.h5


In [3]:
df_map = pd.read_hdf(
    'IGSR/23andme_rel_meta.chip.omni_broad_sanger_combined.20140818.snps.genotypes.h5')
df_ped = pd.read_hdf(
    'IGSR/23andme_rel_ped.chip.omni_broad_sanger_combined.20140818.snps.genotypes.h5')\
        .reset_index()

In [4]:
df_map

,id,chrom,pos,ref,alts,rid,format,format_desc
0,rs3094315,1,752566,G,"(A,)",0,GT,Genotype
1,rs3131972,1,752721,A,"(G,)",0,GT,Genotype
2,rs114525117,1,759036,G,"(A,)",0,GT,Genotype
3,rs12124819,1,776546,A,"(G,)",0,GT,Genotype
4,rs11240777,1,798959,G,"(A,)",0,GT,Genotype
...,...,...,...,...,...,...,...,...
883013,rs41441949,MT,15930,G,"(A,)",24,GT,Genotype
883014,rs41505649,MT,16144,T,"(C,)",24,GT,Genotype
883015,rs41419246,MT,16145,G,"(A,)",24,GT,Genotype
883016,rs41466049,MT,16162,A,"(G,)",24,GT,Genotype


In [5]:
df_ped

id,index,rs3094315,rs3131972,rs114525117,rs12124819,rs11240777,rs6681049,rs116452738,rs72631887,rs4970383,...,rs28357371,rs28357374,rs28357375,rs2853510,rs28357377,rs41441949,rs41505649,rs41419246,rs41466049,rs41479950
0,HG00096,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HG00097,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,HG00098,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,0.0
3,HG00099,1.0,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HG00100,2.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,NA20827,2.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
548,NA20828,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549,NA20829,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
550,NA20831,1.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [6]:
igsr_samples = pd.read_csv("IGSR/igsr_samples.tsv", delimiter="\t")
df_ped = df_ped.merge(igsr_samples, how="left",
                      left_on="index", right_on="Sample name")
df_ped = df_ped[~df_ped['Population code'].isin(["TSI", "IBS", "IBS,MSL"])]
df_ped = df_ped.drop(columns=['Sex', 'Population code', 'Population name',
                              'Superpopulation code', 'Superpopulation name', 'Population elastic ID',
                              'Biosample ID', 'Data collections', 'Sample name'])\
    .reset_index(drop=True)

In [7]:
df_ped

,index,rs3094315,rs3131972,rs114525117,rs12124819,rs11240777,rs6681049,rs116452738,rs72631887,rs4970383,...,rs28357371,rs28357374,rs28357375,rs2853510,rs28357377,rs41441949,rs41505649,rs41419246,rs41466049,rs41479950
0,HG00096,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HG00097,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,HG00098,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.0,0.0
3,HG00099,1.0,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HG00100,2.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,NA12878,2.0,2.0,NaN,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
286,NA12889,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
287,NA12890,1.0,1.0,NaN,0.0,1.0,2.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,NA12891,2.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
worksheet = pd.read_excel("worksheets/variantStats-v2.xlsb.xlsx", sheet_name="all")
# "worksheets/variantStats-v2.xlsb.xlsx

In [9]:
worksheet


,SNPS,REF,ALT,RAF,OR,condition
0,rs12188300,A,T,0.080310,NaN,psoriasis
1,rs13437088,C,T,0.261550,NaN,psoriasis
2,rs4379175,G,T,0.331190,NaN,psoriasis
3,rs17716942,T,C,0.143274,NaN,psoriasis
4,rs6677595,C,T,0.625855,NaN,psoriasis
5,rs12720356,A,C,0.086250,NaN,psoriasis
6,rs582757,C,T,0.724810,NaN,psoriasis
7,rs7552167,A,G,0.860656,NaN,psoriasis
8,rs1295685,A,G,0.804901,NaN,psoriasis
9,rs892085,G,A,0.556606,NaN,psoriasis


In [10]:
with open("variantinfo_vcf/all.json", "r") as f:
    all_json = json.load(f)
assert len(all_json.keys()) >= len(df_map)


In [11]:
def variant_info_from_json(rsid):
    rsid_content = all_json[rsid]
    if False in rsid_content:
        return np.nan, np.nan
    # extract that G>T and take the T as mutation and G as reference
    ref2alt = [(entry[entry.index(">")-1], entry[entry.index(">")+1])
               for entry in rsid_content if ">" in entry]
    if len(ref2alt) == 0:
        return -1, np.nan
    # while mutations could be different the reference should be the same
    # across the board
    if len(set([r[0] for r in ref2alt])) == 1:
        return ref2alt[0][0], list(set([r[1] for r in ref2alt]))
    else:
        return -2, np.nan

In [12]:
tqdm.pandas()
df_map['vi_ref'], df_map['vi_alt'] = zip(
    *df_map['id'].progress_apply(variant_info_from_json))


100%|██████████| 883013/883013 [00:02<00:00, 349146.08it/s]
/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [13]:
# df_map[df_map['vi_ref'] == -1] # TODO rsid's where there isn't >  (2)
# df_map[df_map['vi_ref'] == -2] # TODO rsid's where the ref is different among possibilities (190)
# df_map[df_map['vi_ref'] != df_map['vi_ref']] # TODO rsid's that could not be found in db (94)
df_map = df_map[(df_map['vi_ref'] != -1) & 
                (df_map['vi_ref'] != -2) & 
                (df_map['vi_ref'] == df_map['vi_ref'])]
df_map

,id,chrom,pos,ref,alts,rid,format,format_desc,vi_ref,vi_alt
0,rs3094315,1,752566,G,"(A,)",0,GT,Genotype,G,"[T, A]"
1,rs3131972,1,752721,A,"(G,)",0,GT,Genotype,A,"[C, T, G]"
2,rs114525117,1,759036,G,"(A,)",0,GT,Genotype,G,[A]
3,rs12124819,1,776546,A,"(G,)",0,GT,Genotype,A,[G]
4,rs11240777,1,798959,G,"(A,)",0,GT,Genotype,G,[A]
...,...,...,...,...,...,...,...,...,...,...
883013,rs41441949,MT,15930,G,"(A,)",24,GT,Genotype,G,"[C, A]"
883014,rs41505649,MT,16144,T,"(C,)",24,GT,Genotype,T,[C]
883015,rs41419246,MT,16145,G,"(A,)",24,GT,Genotype,G,"[C, A]"
883016,rs41466049,MT,16162,A,"(G,)",24,GT,Genotype,A,"[T, G]"


In [14]:
def alt_in_vinfo(x):
    assert type(x[1]) is tuple
    alt_differs = False
    ref_differs = False
    for alt in x[1]:
        if alt not in x[3]:
            alt_differs = True
    if x[0] != x[2]:
        ref_differs = True
    if alt_differs and not ref_differs:
        return 1
    if ref_differs and not alt_differs:
        return 2
    if ref_differs and alt_differs:
        return 3
    return 0

df_map['present'] = df_map[['ref','alts', 'vi_ref','vi_alt']].progress_apply(alt_in_vinfo, axis=1)


100%|██████████| 882550/882550 [00:08<00:00, 106597.09it/s]
<ipython-input-14-85a4a823058f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_map['present'] = df_map[['ref','alts', 'vi_ref','vi_alt']].progress_apply(alt_in_vinfo, axis=1)


In [73]:
# for snp in df_map[df_map['present'] != 0]['id'].tolist():
#     if snp in wksht_map['SNPS'].tolist():
#         print(snp)
# df_map[df_map['present'] != 0]['id'].tolist()


In [15]:
df_map = df_map[~df_map['id'].isin(df_map[df_map['present'] != 0]['id'].tolist())]
df_map

,id,chrom,pos,ref,alts,rid,format,format_desc,vi_ref,vi_alt,present
0,rs3094315,1,752566,G,"(A,)",0,GT,Genotype,G,"[T, A]",0
1,rs3131972,1,752721,A,"(G,)",0,GT,Genotype,A,"[C, T, G]",0
2,rs114525117,1,759036,G,"(A,)",0,GT,Genotype,G,[A],0
3,rs12124819,1,776546,A,"(G,)",0,GT,Genotype,A,[G],0
4,rs11240777,1,798959,G,"(A,)",0,GT,Genotype,G,[A],0
...,...,...,...,...,...,...,...,...,...,...,...
883013,rs41441949,MT,15930,G,"(A,)",24,GT,Genotype,G,"[C, A]",0
883014,rs41505649,MT,16144,T,"(C,)",24,GT,Genotype,T,[C],0
883015,rs41419246,MT,16145,G,"(A,)",24,GT,Genotype,G,"[C, A]",0
883016,rs41466049,MT,16162,A,"(G,)",24,GT,Genotype,A,"[T, G]",0


In [16]:
# inspect this for differences in REF/AT and vi_ref and vi_alt
worksheet.merge(df_map, left_on="SNPS", right_on='id', how='inner')


,SNPS,REF,ALT,RAF,OR,condition,id,chrom,pos,ref,alts,rid,format,format_desc,vi_ref,vi_alt,present
0,rs13437088,C,T,0.261550,NaN,psoriasis,rs13437088,6,31355119,C,"(T,)",5,GT,Genotype,C,[T],0
1,rs17716942,T,C,0.143274,NaN,psoriasis,rs17716942,2,163260691,T,"(C,)",1,GT,Genotype,T,[C],0
2,rs6677595,C,T,0.625855,NaN,psoriasis,rs6677595,1,152590187,C,"(T,)",0,GT,Genotype,C,"[T, G, A]",0
3,rs12720356,A,C,0.086250,NaN,psoriasis,rs12720356,19,10469975,A,"(C,)",18,GT,Genotype,A,"[C, G]",0
4,rs7552167,A,G,0.860656,NaN,psoriasis,rs7552167,1,24518643,A,"(G,)",0,GT,Genotype,A,"[T, G]",0
5,rs1295685,A,G,0.804901,NaN,psoriasis,rs1295685,5,131996445,A,"(G,)",4,GT,Genotype,A,"[T, G]",0
6,rs892085,G,A,0.556606,NaN,psoriasis,rs892085,19,10818092,G,"(A,)",18,GT,Genotype,G,[A],0
7,rs8016947,T,G,0.560449,NaN,psoriasis,rs8016947,14,35832666,T,"(G,)",13,GT,Genotype,T,"[C, G]",0
8,rs12445568,T,C,0.363960,NaN,psoriasis,rs12445568,16,31004812,T,"(C,)",15,GT,Genotype,T,[C],0
9,rs4561177,A,G,0.406626,NaN,psoriasis,rs4561177,11,109962432,A,"(G,)",10,GT,Genotype,A,"[C, T, G]",0


In [17]:
import pickle
# NOTE only picking variants that are consistent b/w spreadsheet and db
#signature = worksheet.merge(df_map, left_on="SNPS", right_on=1, how='inner').iloc[[0,1,4,7,8,9,10,11,14,17],:]#.sample(n=12,replace=False)
wksht_map = worksheet.merge(df_map, left_on="SNPS", right_on='id', how='inner')
wksht_map = wksht_map.drop(wksht_map.index[[32]])\
    .drop_duplicates(subset=['SNPS'])\
    .reset_index(drop=True)
# wksht_map = wksht_map.drop(wksht_map.index[[42]]).reset_index(drop=True)
# wksht_map['orig_index'] = wksht_map['index']
# wksht_map['index'] = 2*wksht_map['index']+6
# rs12295535
wksht_map

# 42 SNPs available from the hapmap3 data; only 27 available here

# 12 * 24 checks


,SNPS,REF,ALT,RAF,OR,condition,id,chrom,pos,ref,alts,rid,format,format_desc,vi_ref,vi_alt,present
0,rs13437088,C,T,0.261550,NaN,psoriasis,rs13437088,6,31355119,C,"(T,)",5,GT,Genotype,C,[T],0
1,rs17716942,T,C,0.143274,NaN,psoriasis,rs17716942,2,163260691,T,"(C,)",1,GT,Genotype,T,[C],0
2,rs6677595,C,T,0.625855,NaN,psoriasis,rs6677595,1,152590187,C,"(T,)",0,GT,Genotype,C,"[T, G, A]",0
3,rs12720356,A,C,0.086250,NaN,psoriasis,rs12720356,19,10469975,A,"(C,)",18,GT,Genotype,A,"[C, G]",0
4,rs7552167,A,G,0.860656,NaN,psoriasis,rs7552167,1,24518643,A,"(G,)",0,GT,Genotype,A,"[T, G]",0
5,rs1295685,A,G,0.804901,NaN,psoriasis,rs1295685,5,131996445,A,"(G,)",4,GT,Genotype,A,"[T, G]",0
6,rs892085,G,A,0.556606,NaN,psoriasis,rs892085,19,10818092,G,"(A,)",18,GT,Genotype,G,[A],0
7,rs8016947,T,G,0.560449,NaN,psoriasis,rs8016947,14,35832666,T,"(G,)",13,GT,Genotype,T,"[C, G]",0
8,rs12445568,T,C,0.363960,NaN,psoriasis,rs12445568,16,31004812,T,"(C,)",15,GT,Genotype,T,[C],0
9,rs4561177,A,G,0.406626,NaN,psoriasis,rs4561177,11,109962432,A,"(G,)",10,GT,Genotype,A,"[C, T, G]",0


In [18]:
missing_snp = list(
    set(worksheet['SNPS'].tolist()) - set(wksht_map['SNPS'].tolist()))
print(missing_snp)
print(len(missing_snp))
with open("IGSR/missing_snp.pkl", "wb") as f:
    pickle.dump(missing_snp, f)
#wksht_map['SNPS']


['rs545979', 'rs1250546', 'rs1665050', 'rs645078', 'rs582757', 'rs1327914', 'rs847', 'rs4520482', 'rs1056198', 'rs12188300', 'rs10738626', 'rs10994675', 'rs4379175']
13


In [20]:
# filter down the ped now
col_list = df_map['id'].tolist() + ['index']
df_ped = df_ped[col_list]

In [21]:
# hopefully prints nothing!
for col in wksht_map['SNPS'].tolist():
    if col not in col_list:
        print(col)

In [22]:
#np.random.seed(2022)
df_ped_bootstrap = pd.concat(
    [df_ped, df_ped.sample(n=len(df_ped),replace=True), df_ped.sample(n=len(df_ped),replace=True)]) 

In [23]:
#df_ped_bootstrap # 165 * 3 rows 
from sklearn.model_selection import train_test_split
#np.random.seed(2022)
df_cases, df_control = train_test_split(df_ped_bootstrap, test_size=0.5)
df_cases['group'] = "cases"
df_control['group'] = "control"
df_cases = df_cases.reset_index(drop=True)
df_control = df_control.reset_index(drop=True)
# 247 + 248 = 495


<ipython-input-23-d12e1eed16b0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cases['group'] = "cases"
<ipython-input-23-d12e1eed16b0>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_control['group'] = "control"


In [24]:
print(len(df_cases),len(df_control))

435 435


In [25]:
num_blocks = 10
block_size = len(df_cases) // num_blocks
blocks = np.arange(0, len(df_cases), block_size)
blocks[-1] = len(df_cases) # make the last one go all the way
blocks = np.diff(blocks)
blocks


array([43, 43, 43, 43, 43, 43, 43, 43, 43, 48])

In [26]:
# np.random.seed(2022)
df_cases_blocks = []
cases_index_bag = list(range(len(df_cases)))
for block in tqdm(blocks): 
    indices = np.random.choice(cases_index_bag, size=block, replace=False)
    df_cases_blocks.append(df_cases.iloc[indices, :].copy(deep=True))
    # remove from the bag
    for selected in indices:
        cases_index_bag.remove(selected)
# bag should be empty when this is done 
assert len(cases_index_bag) == 0
# parts should sum to the total 
assert sum([len(df) for df in df_cases_blocks]) == len(df_cases)

os.makedirs("gen_info_vcf",exist_ok=True)
with open("gen_info_vcf/blocks.info", "w") as f:
    for index, block in enumerate(df_cases_blocks):
        f.write(f"----block {index}\n")
        f.write(f"{list(block.index)}\n")
df_cases_blocks = [df.reset_index(drop=True) for df in df_cases_blocks]

100%|██████████| 10/10 [00:02<00:00,  4.57it/s]


In [27]:
print([len(b) for b in df_cases_blocks])
sum([len(b) for b in df_cases_blocks])


[43, 43, 43, 43, 43, 43, 43, 43, 43, 48]


435

In [28]:
# each block is associated with a different signature 
np.random.seed(2022)
sig_list = []
rsid_contribution = 10
for block_num in list(range(num_blocks)):
    sig_list.append(wksht_map.sample(n=rsid_contribution, replace=False))

with open("gen_info_vcf/blocks.info", "a") as f:
    for index,sig in enumerate(sig_list):
        f.write(f"----block {index}\n")
        f.write(f"{list(sig['SNPS'])}\n")
sig_list = [df.reset_index(drop=True) for df in sig_list]
len(sig_list)

10

In [29]:
sig_list[1]

,SNPS,REF,ALT,RAF,OR,condition,id,chrom,pos,ref,alts,rid,format,format_desc,vi_ref,vi_alt,present
0,rs12720356,A,C,0.086250,NaN,psoriasis,rs12720356,19,10469975,A,"(C,)",18,GT,Genotype,A,"[C, G]",0
1,rs892085,G,A,0.556606,NaN,psoriasis,rs892085,19,10818092,G,"(A,)",18,GT,Genotype,G,[A],0
2,rs479844,A,G,0.558801,NaN,atopic dermtitis,rs479844,11,65551957,A,"(G,)",10,GT,Genotype,A,"[C, G]",0
3,rs4561177,A,G,0.406626,NaN,psoriasis,rs4561177,11,109962432,A,"(G,)",10,GT,Genotype,A,"[C, T, G]",0
4,rs7130588,A,G,0.335615,NaN,atopic dermtitis,rs7130588,11,76270683,A,"(G,)",10,GT,Genotype,A,[G],0
5,rs17716942,T,C,0.143274,NaN,psoriasis,rs17716942,2,163260691,T,"(C,)",1,GT,Genotype,T,[C],0
6,rs4821544,T,C,0.325380,NaN,atopic dermtitis,rs4821544,22,37258503,T,"(C,)",21,GT,Genotype,T,"[C, A]",0
7,rs7552167,A,G,0.860656,NaN,psoriasis,rs7552167,1,24518643,A,"(G,)",0,GT,Genotype,A,"[T, G]",0
8,rs6677595,C,T,0.625855,NaN,psoriasis,rs6677595,1,152590187,C,"(T,)",0,GT,Genotype,C,"[T, G, A]",0
9,rs12445568,T,C,0.363960,NaN,psoriasis,rs12445568,16,31004812,T,"(C,)",15,GT,Genotype,T,[C],0


In [30]:
from collections import defaultdict
def flip(p):
    return True if random.random() < p else False


In [31]:
sum([len(b) for b in df_cases_blocks])
block_lens = [len(b) for b in df_cases_blocks]
block_lens


[43, 43, 43, 43, 43, 43, 43, 43, 43, 48]

In [32]:
df_cases_blocks[0]

,rs3094315,rs3131972,rs114525117,rs12124819,rs11240777,rs6681049,rs116452738,rs72631887,rs4970383,rs28678693,...,rs28357375,rs2853510,rs28357377,rs41441949,rs41505649,rs41419246,rs41466049,rs41479950,index,group
0,2.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12891,cases
1,1.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00116,cases
2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00114,cases
3,1.0,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12275,cases
4,2.0,2.0,NaN,1.0,0.0,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,HG01334,cases
5,2.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12400,cases
6,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,HG00136,cases
7,2.0,2.0,0.0,NaN,0.0,2.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NA10839,cases
8,2.0,2.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12144,cases
9,1.0,1.0,NaN,0.0,1.0,2.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12890,cases


In [33]:
#df_cases_candidates
ests = defaultdict(list)
# np.random.seed(2022)
assert len(df_cases_blocks) == len(sig_list)
# visit each block
for b_ind in tqdm(range(len(df_cases_blocks)),total=len(df_cases_blocks)):
    # visit the signature associated with that block
    for snp in zip(sig_list[b_ind]['ALT'], sig_list[b_ind]['RAF'], 
                   sig_list[b_ind]['SNPS']):
        # simulate introduction of variant for each individual
        # in that block according to RAF 
        num_het_edits = 0
        num_hom_edits = 0
        assert len(df_cases_blocks[b_ind]) == block_lens[b_ind]
        for i in range(len(df_cases_blocks[b_ind])):
            if flip(snp[1]):
                df_cases_blocks[b_ind].at[i, snp[2]] = 1
                num_het_edits += 1
                # no more than half the chance of RAF to introduce
                # a homozygous variant 
                if flip(snp[1]/2):
                    df_cases_blocks[b_ind].at[i, snp[2]] = 2
                    num_hom_edits += 1
        assert len(df_cases_blocks[b_ind]) == block_lens[b_ind]
        ests[snp[-1]].append((num_het_edits / len(df_cases_blocks[b_ind]), 
                    snp[1], 
                    num_hom_edits / len(df_cases_blocks[b_ind]),
                    snp[1]/2))

100%|██████████| 10/10 [00:00<00:00, 172.77it/s]


In [21]:
#ests


In [34]:
df_cases_mutated = pd.concat(df_cases_blocks).reset_index(drop=True)
df_cases_mutated


,rs3094315,rs3131972,rs114525117,rs12124819,rs11240777,rs6681049,rs116452738,rs72631887,rs4970383,rs28678693,...,rs28357375,rs2853510,rs28357377,rs41441949,rs41505649,rs41419246,rs41466049,rs41479950,index,group
0,2.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12891,cases
1,1.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00116,cases
2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00114,cases
3,1.0,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12275,cases
4,2.0,2.0,NaN,1.0,0.0,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,HG01334,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NA10855,cases
431,1.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00134,cases
432,1.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NA10861,cases
433,2.0,2.0,0.0,NaN,0.0,2.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NA10830,cases


In [35]:
# colid2val = {}
val2count = defaultdict(int)
df_cols = list(df_cases_mutated.columns)
for i in tqdm(range(0, 882522-2), total=len(range(0, 882522-2))):
    z = list(zip(list(df_cases_mutated[df_cols[i]].value_counts().index),
                 list(df_cases_mutated[df_cols[i]].value_counts().values)))
    for i,v in z:
        val2count[i] += v



100%|██████████| 882520/882520 [06:14<00:00, 2359.63it/s]


In [36]:
val2count2 = defaultdict(int)
# series_list = []
df_cols = list(df_cases.columns)
for i in tqdm(range(0, 882522-2), total=len(range(0, 882522-2))):
    z = list(zip(list(df_cases[df_cols[i]].value_counts().index), 
                 list(df_cases[df_cols[i]].value_counts().values)))
    for i,v in z:
        val2count2[i] += v
    # z = [list(i[0]*i[1]) for i in z]
    # series_list.append(pd.Series([item for sublist in z for item in sublist]))
# b = pd.concat(series_list)


100%|██████████| 882520/882520 [06:17<00:00, 2340.51it/s]


In [39]:
val2count


defaultdict(int, {2.0: 66938125, 1.0: 103362239, 0.0: 203445056, 4.0: 94})

In [40]:
val2count2


defaultdict(int, {2.0: 66938290, 1.0: 103361708, 0.0: 203445410, 4.0: 94})

In [41]:
for k, v in zip(val2count.items(), val2count2.items()):
    assert k[0] == v[0]
    print((k[0], k[1] - v[1]))


(2.0, -165)
(1.0, 531)
(0.0, -354)
(4.0, 0)


In [42]:
del df_ped_bootstrap
del all_json

In [43]:
df_together = pd.concat([df_cases_mutated, df_control]).reset_index(drop=True)
# df_together = df_together[['group'] +
#                  [col for col in df_together.columns if col != 'group']]
df_together
#assert len(df_together) == len(df_ped_bootstrap)


,rs3094315,rs3131972,rs114525117,rs12124819,rs11240777,rs6681049,rs116452738,rs72631887,rs4970383,rs28678693,...,rs28357375,rs2853510,rs28357377,rs41441949,rs41505649,rs41419246,rs41466049,rs41479950,index,group
0,2.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12891,cases
1,1.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00116,cases
2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00114,cases
3,1.0,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12275,cases
4,2.0,2.0,NaN,1.0,0.0,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,HG01334,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,2.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NA12708,control
866,1.0,1.0,NaN,1.0,1.0,2.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00127,control
867,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12006,control
868,2.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12400,control


In [44]:
del df_cases_mutated
del df_control
del df_cases_blocks

In [45]:
df_together.to_hdf("df_together_vcf.h5", key='db', mode='w', index=False)
df_map.to_hdf("df_together_vcf_map.h5", key='db', mode='w', index=False)

/home/graph/jbonnell/miniconda3/envs/blueberry/lib/python3.9/site-packages/pandas/core/generic.py:2605: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['id', 'chrom', 'ref', 'alts', 'format', 'format_desc', 'vi_ref',
       'vi_alt'],
      dtype='object')]

  pytables.to_hdf(


### post recoding

In [18]:
df_ped = pd.read_hdf('df_together_vcf.h5')
df_map = pd.read_hdf("df_together_vcf_map.h5")
worksheet = pd.read_excel(
    "worksheets/variantStats-v2.xlsb.xlsx", sheet_name="all").\
    drop_duplicates(subset=['SNPS'])


In [19]:
df_map

,id,chrom,pos,ref,alts,rid,format,format_desc,vi_ref,vi_alt,present
0,rs3094315,1,752566,G,"(A,)",0,GT,Genotype,G,"[T, A]",0
1,rs3131972,1,752721,A,"(G,)",0,GT,Genotype,A,"[C, T, G]",0
2,rs114525117,1,759036,G,"(A,)",0,GT,Genotype,G,[A],0
3,rs12124819,1,776546,A,"(G,)",0,GT,Genotype,A,[G],0
4,rs11240777,1,798959,G,"(A,)",0,GT,Genotype,G,[A],0
...,...,...,...,...,...,...,...,...,...,...,...
883013,rs41441949,MT,15930,G,"(A,)",24,GT,Genotype,G,"[C, A]",0
883014,rs41505649,MT,16144,T,"(C,)",24,GT,Genotype,T,[C],0
883015,rs41419246,MT,16145,G,"(A,)",24,GT,Genotype,G,"[C, A]",0
883016,rs41466049,MT,16162,A,"(G,)",24,GT,Genotype,A,"[T, G]",0


In [20]:
# TODO cases where there was a "2" in each allele, therefore 2+2 = 4
# let's delete those columns
weird_cols = []
for col in tqdm(list(df_ped.columns)):
    if sum(df_ped[col].isin([4])) > 0:
        weird_cols.append(col)

100%|██████████| 882522/882522 [02:18<00:00, 6381.78it/s]


In [21]:
print(weird_cols)
# ['rs34305730', 'rs77338688', 'rs2283149', 'rs74532824']

['rs34305730', 'rs77338688', 'rs2283149', 'rs74532824']


In [22]:
# for loop shouldn't print anything
for weird in weird_cols:
    if weird in worksheet['SNPS'].tolist():
        print(weird)

In [23]:
print(len(df_ped.columns))
df_ped = df_ped.drop(columns=weird_cols)
print(len(df_ped.columns))

882522
882518


In [24]:
worksheet[worksheet['SNPS'].isin(list(worksheet[worksheet.duplicated(subset=['SNPS'])]['SNPS']))]
# empty, please!

,SNPS,REF,ALT,RAF,OR,condition


In [25]:
to_drop = []
df_cols = list(df_ped.columns)
for i in tqdm(range(0, len(df_cols))):
    #3 in set(recoded_df[i].value_counts().index):
    if df_ped[df_cols[i]].isna().sum().sum() > 0: #or \
        to_drop.append(df_cols[i])

100%|██████████| 882518/882518 [01:51<00:00, 7935.53it/s]


In [26]:
len(to_drop)

385049

In [27]:
df_ped
# TODO sex distribution 50/50?


,rs3094315,rs3131972,rs114525117,rs12124819,rs11240777,rs6681049,rs116452738,rs72631887,rs4970383,rs28678693,...,rs28357375,rs2853510,rs28357377,rs41441949,rs41505649,rs41419246,rs41466049,rs41479950,index,group
0,2.0,2.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12891,cases
1,1.0,1.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00116,cases
2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00114,cases
3,1.0,1.0,NaN,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12275,cases
4,2.0,2.0,NaN,1.0,0.0,2.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,HG01334,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,2.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NA12708,control
866,1.0,1.0,NaN,1.0,1.0,2.0,0.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HG00127,control
867,2.0,2.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12006,control
868,2.0,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NA12400,control


In [28]:
to_drop[:10]

['rs3094315',
 'rs3131972',
 'rs114525117',
 'rs12124819',
 'rs116452738',
 'rs72631887',
 'rs4970383',
 'rs28678693',
 'rs7537756',
 'rs1110052']

In [31]:
with open("gen_info_vcf/blocks.info", "r") as f:
    sig_info = [eval(l.strip()) for l in f.readlines()[20:] if "----" not in l]
sig_info = set([item for sublist in sig_info for item in sublist])
#sig_info

In [31]:
#sig_info


In [32]:
sig_positions = list(df_map[df_map['id'].isin(sig_info)].merge(worksheet, how='inner', left_on='id', right_on='SNPS')[['id','condition']].itertuples(index=False, name=None))
#sig_positions = list(df_map[df_map[1].isin(sig_info)])
sig_positions


[('rs7552167', 'psoriasis'),
 ('rs6661961', 'atopic dermtitis'),
 ('rs6677595', 'psoriasis'),
 ('rs10865331', 'psoriasis'),
 ('rs759382', 'atopic dermtitis'),
 ('rs2111485', 'psoriasis'),
 ('rs17716942', 'psoriasis'),
 ('rs1295686', 'atopic dermtitis'),
 ('rs1295685', 'psoriasis'),
 ('rs848', 'atopic dermtitis'),
 ('rs9504361', 'psoriasis'),
 ('rs13437088', 'psoriasis'),
 ('rs12153855', 'atopic dermtitis'),
 ('rs176095', 'atopic dermtitis'),
 ('rs11795343', 'psoriasis'),
 ('rs12295535', 'atopic dermtitis'),
 ('rs11033603', 'atopic dermtitis'),
 ('rs479844', 'atopic dermtitis'),
 ('rs7130588', 'atopic dermtitis'),
 ('rs7927894', 'atopic dermtitis'),
 ('rs4561177', 'psoriasis'),
 ('rs8016947', 'psoriasis'),
 ('rs2041733', 'atopic dermtitis'),
 ('rs12445568', 'psoriasis'),
 ('rs4796793', 'atopic dermtitis'),
 ('rs16948048', 'atopic dermtitis'),
 ('rs11652075', 'psoriasis'),
 ('rs12720356', 'psoriasis'),
 ('rs892085', 'psoriasis'),
 ('rs4821544', 'atopic dermtitis')]

In [34]:
assert len(sig_positions) == len(set(sig_positions))
len(sig_positions)

30

In [35]:
with open("gen_info_vcf/sig.info", "w") as f:
    f.writelines([f"{l[0]} {l[1]}\n" for l in sig_positions])

In [36]:
#signature[signature['SNPS'].isin(to_drop_snps)]
marked_sigs = list(sig_info.intersection(to_drop))
print(len(marked_sigs))
marked_sigs # 18


11


['rs10865331',
 'rs11795343',
 'rs11652075',
 'rs2111485',
 'rs1295685',
 'rs8016947',
 'rs16948048',
 'rs9504361',
 'rs848',
 'rs4561177',
 'rs2041733']

In [37]:
set(list(df_map[df_map['id'].isin(marked_sigs)]['id']))


{'rs10865331',
 'rs11652075',
 'rs11795343',
 'rs1295685',
 'rs16948048',
 'rs2041733',
 'rs2111485',
 'rs4561177',
 'rs8016947',
 'rs848',
 'rs9504361'}

In [38]:
# don't drop anything that was part of a signature
#to_drop = list(set(to_drop) - set(list(signature[signature['SNPS'].isin(to_drop_snps)]['index'])))
to_drop = list(
    set(to_drop) - set(list(df_map[df_map['id'].isin(marked_sigs)]['id'])))


In [40]:
len(to_drop)  # 385038


385038

In [41]:
df_ped = df_ped.drop(to_drop, axis=1)


In [42]:
df_ped
# 497480


,rs11240777,rs6681049,rs4475691,rs72631889,rs13302982,rs2880024,rs2272757,rs13303010,rs3935066,rs6669800,...,rs2552791,rs3865913,rs35572756,rs9943032,rs7067398,rs35557642,rs9786255,rs41531144,index,group
0,1.0,2.0,0.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,NA12891,cases
1,2.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,HG00116,cases
2,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,HG00114,cases
3,1.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,NA12275,cases
4,0.0,2.0,1.0,0.0,2.0,1.0,0.0,2.0,2.0,2.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,HG01334,cases
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,1.0,2.0,1.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,...,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,NA12708,control
866,1.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,HG00127,control
867,1.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,NA12006,control
868,0.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,NA12400,control


In [43]:
set([c for c in list(df_ped.columns) if c in set(worksheet['SNPS'].tolist())]) - set(sig_info)


{'rs10994675'}

In [44]:
df_ped.to_hdf("recoded_hapmap_ped_case_control_vcf.h5", key="db", mode="w")